<a href="https://colab.research.google.com/github/ginttone/multi_deeplearning/blob/master/NLP/SentencePiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDb_Reviews.csv 리뷰 긍정 부정

In [ ]:
!python -m pip install sentencepiece

In [3]:
import sentencepiece as spm

In [6]:
!curl -O https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62.8M  100 62.8M    0     0   146M      0 --:--:-- --:--:-- --:--:--  146M


In [7]:
import pandas as pd

In [9]:
df_data = pd.read_csv('./IMDb_Reviews.csv')
df_data.head(3)

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0


In [12]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [15]:
# 실습을 위해 데이터 쪼갬
df_data= df_data[0:2000]

* text로 line by line으로 쭉 저장

판다스의 with

with ㅇㅇㅇ as ㅁㅁㅁ

with는 컴퓨터의 자원관리를 자동적으로 해준다. db관련은 with으로 가둬주는 경우가 많다. 



In [16]:
with open('imdb_review.txt','w') as f:
  f.write('\n'.join(df_data['review']))

## sentence piece만들기

(통계에 의한전통머신러닝의 일부기능 사용)

input은 교육하기 위해 필요한 텍스트 집합

model_prefix는 교육하고나서 센텐스피스가 만들어지면 그 이름 

vocab_size는 보케블러리 사이즈(단어갯수) 몇개까지 할건지

model_type = bpe는 중심단어를 기준으로 주변단어 찾는데 어떻게 찾을지에 대한 알고리즘

- 실행하면 imdb.model, imdb.vocab 파일이 생긴다.

In [17]:
spm.SentencePieceTrainer.Train('--input=./imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

텝으로 나누어져(sep) 있어 \t함

In [19]:
import csv

단어뒤에 붙는 숫자가 그 단어의 유니크한 값이 된다

단어 앞에 언더바 `_`가 들어가는데 , 이 의미는 _로 해서 앞의 단어와 뒷단어의 연관성을 측정한다.

In [22]:
df_vocab = pd.read_csv('./imdb.vocab',sep='\t',quoting=csv.QUOTE_NONE , header=None)
df_vocab.sample(10)

,0,1
1052,anc,-1049
1014,uck,-1011
165,▁pl,-162
2679,▁revol,-2676
1245,uly,-1242
4962,C,-4959
3462,reg,-3459
2359,angu,-2356
144,ght,-141
3038,How,-3035


문장에서 단어에 대한 구분은 하는데 그 단어에대해 특정기호를 인위적으로 넣어서 해석하는데 도움을 주는 것

예 ) `<unk>`

In [23]:
df_vocab.head(7)

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,▁t,0
4,▁a,-1
5,he,-2
6,in,-3


load_model과 유사한 .load()

In [24]:
spp = spm.SentencePieceProcessor()
spp.load('./imdb.model')

True

In [25]:
line01 = "I didn't at all think of it this way."

line01문장이 토크나이징 되어 아래와 같이 변화가 일어난다.

encode_as_pieces(line01)

`_`는 스페이스 라고 생각하면 쉽다.

In [27]:
print(spp.encode_as_pieces(line01))

['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']


line01문장을 보케브러리 기준으로 숫자로 바꿔준다

encode_as_ids(line01)

In [28]:
print(spp.encode_as_ids(line01))

[40, 588, 4949, 4926, 136, 175, 400, 30, 56, 77, 418, 4945]


이것을 model교육을 시킨다. 

교육 후 분류 모델을 만들어낸다.